In [1]:
import pandas as pd
import numpy as np

In [2]:
issues = pd.read_json("preprocessed.json")
issues.sample(5)

,repo_name,title,body,languages,tags
13,systemd,systemd-machined can be crashed by calling the...,### systemd version the issue has been seen wi...,"[C, Python, Shell]",systemdmachine crash call list varlink method ...
12,systemd,memory.swap.max == max contrary to systemctl s...,### systemd version the issue has been seen wi...,"[C, Python, Shell]",memoryswapmax max contrary systemctl showexpec...
19,prisma,Inconsistent column data: Could not convert va...,### Bug description\n\nWe recently changed dat...,"[TypeScript, JavaScript, Shell]",inconsistent column datum convert value false ...
8,llama-models,lm_head weight of Llama3.2_3B_instruct model,"Hello, I find that theres no lm_head weight in...",[Python],lmhead weight llamabinstruct model hello find ...
7,llama-models,Can llama3.2 vision accept no image?,"Hello, \r\n\r\nthank you for thegreat work.\r\...",[Python],llama vision accept image hello thank thegreat...


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
class Litmus:
    def __init__(self, data):
        self.data = data
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.tfidf_matrix = self.vectorizer.fit_transform(self.data['tags'])
        
    def get_rec(self, lang, limit=5):
            language_mask = self.data['languages'].apply(
                lambda x: lang.lower() in str(x).lower()
            )
            if not any(language_mask):
                return []

            relevant_issues = self.data[language_mask]
            relevant_matrix = self.tfidf_matrix[language_mask]
            
            similarity_scores = cosine_similarity(relevant_matrix, relevant_matrix)
            
            recommendations = []
            seen_titles = set()
            
            for idx in range(len(relevant_issues)):
                similar_indices = similarity_scores[idx].argsort()[::-1][1:limit+1]
            
                for similar_idx in similar_indices:
                    issue = relevant_issues.iloc[similar_idx]
                    
                    # no dups!
                    if issue['title'] not in seen_titles:
                        recommendations.append({
                            'repo_name': issue['repo_name'],
                            'title': issue['title'],
                            'languages': issue['languages'],
                            'tags': issue['tags']
                        })
                        seen_titles.add(issue['title'])
                        
                    if len(recommendations) >= limit:
                        break
                        
                if len(recommendations) >= limit:
                    break
                
            return recommendations

In [5]:
recommender = Litmus(issues)
recommendations = recommender.get_rec('Java', limit=3)
for i, rec in enumerate(recommendations, 1):
        print(f"\nRecommendation {i}:")
        print(f"Repository: {rec['repo_name']}")
        print(f"Title: {rec['title']}")
        print(f"Languages: {rec['languages']}")


Recommendation 1:
Repository: vercel
Title: [cli] Add telemetry for `vercel teams invite email1, email2`
Languages: ['TypeScript', 'JavaScript', 'HTML']

Recommendation 2:
Repository: spring-data-mongodb
Title: Cannot add litteral array with SetUnion
Languages: ['Java', 'Kotlin', 'Dockerfile']

Recommendation 3:
Repository: commitlint
Title: fix: <title> type-enum inside RuleConfigSeverity.Error is not working.
Languages: ['TypeScript', 'JavaScript']
